In [50]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm
#from google.colab import drive
import os
from matplotlib import style
style.use('fivethirtyeight')
from fuzzywuzzy import fuzz, process
import numpy as np


In [51]:
lq =  pd.read_csv('bcstores.csv')
print('The shape: %d x %d' % lq.shape)
print('There are %d unique values' % lq.name.shape[0])

The shape: 500 x 8
There are 500 unique values


In [52]:
beer = pd.read_csv('/home/sam/Downloads/beeradvocate.csv')

In [53]:
beer['comb'] = beer.mnf.str.lower()+ ' '+ beer.name.str.lower()

beer.comb = beer.comb.str.lower().str.replace("brewing","").str.replace("brewery","")

In [54]:
beer['comb']

0                                                         four winds brewing co. nectarous
1                                                    driftwood brewery sartori harvest ipa
2                                                            driftwood brewery fat tug ipa
3                         central city brewers + distillers red racer ipa (india pale ale)
4                                                            driftwood brewery singularity
                                              ...                                         
508                                      brasseries kronenbourg kronenbourg 1664 blanc 0.5
509                       anheuser-busch labatt - budweiser prohibition brew non-alcoho...
510                                                 anheuser-busch labatt - budweiser zero
511    central city brewers + distillers red racer - street legal dealcoholized ipa tal...
512                                           warsteiner brauerei warsteiner premium fresh

In [55]:
pd.set_option('display.max_rows',500)

# Drop duplicates: diff packing, diff type( can or bottle)

In [56]:
lq.name = lq.name.str.lower().str.replace(" tall can","").str.replace(" can","").str.replace(" 4-pack","").str.replace(" 6-pack","")

In [57]:
lq = lq.sort_values(['name','price']).drop_duplicates(subset = ['name'], keep = 'first')

In [58]:
lq

,name,country,price,sku_link,style,alc,beer_type,desc
0,33 acres of life lager,Canada,$12.29,/product/772335,California Common Or Steam,4.8%,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulness and persistence, the California Common was born. Easy on the alcohol and weighing in on the taste for a complete balance. Fruit like quality is created by fermenting the lager at ale temperatures. Hints at a full-bodied ale yet retains a crisp finish.\xa0\n"
1,33 acres of nirvana 6 pack,Canada,$13.49,/product/880930,India Pale Ale,7%,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved craft beer brewers. This West Coast Indian Pale Ale beautifully balances bitter-fresh pine notes with grapefruit rind and candied orange citrus. Try it with some fried chicken for a true taste of nirvana.\n
2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,Pale Ale,5.6%,Domestic Beer - BC Craft Beer,"A full flavoured beer integrated with a distinct floral hop gives a quality unique to our Pacific Northwest surroundings. This style is evolutionary from a IPA with an assertive hop, lower alcohol content, and slight pine aroma. Low in malt, this beer has a refined quality and a refreshing temperament.\n"
4,33 acres of sunshine,Canada,$21.99,/product/23859,Wheat,5%,Domestic Beer - BC Craft Beer,no info
3,33 acres of sunshine blanche,Canada,$12.29,/product/90837,Wheat,5%,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this romantic French Blanchè can willingly co-star any seasons current scenes. Showcasing flare for the prismatic, 33 Acres of Sunshine synthesizes an unfiltered wheat dream. Joining the arrays, glowing hops of Aurora and Styrian Golding awaken subtle anise seeds, dazzling coriander, and candied orange ovations.\xa0\n"
6,alexander keith's - india pale ale,Canada,$14.99,/product/687251,India Pale Ale,5%,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale is light in colour and hopped in flavour. Only the lightest and finest barleys that produce a pale malt are used while the amounts of hops are increased so as to give a pronounced hop flavour.\n"
7,alhambra reserva 1925,Spain,$13.29,/product/638759,Pilsner,6.4%,Import Beer,This award-winning lager is brewed over 35 days. The nose is full-on caramel leading to a full-flavoured palate and a satisfying finish.\n
8,asahi super dry,Thailand,$14.79,/product/439950,Pilsner,5%,Import Beer,no info
9,asahi super dry big,Japan,$14.79,/product/541201,Pilsner,5%,Import Beer,no info
13,balderdash brewing - sneaky weasel craft lager,Canada,$1.99,/product/198948,Lager,5.6%,Domestic Beer - BC Craft Beer,"Now in a tall can! This mildly hopped craft lager delivers a bold 5.6% alc/vol, but goes down crisp and clean. Pairs well with game day celebrations, fun in the sun and good times with friends.\n"


# Drop variety pack

In [59]:
lq.drop(labels = lq[lq['style'].str.contains('Variety')].index, inplace = True)

In [60]:
lq.drop(labels = lq[lq['name'].str.contains('variety')].index, inplace = True)

In [61]:
def split_str(string):
    return set(string.split(" "))

In [62]:
lq.desc = lq.desc.apply(lambda x: x.replace("\\n","").replace("\\xa0",""))

In [63]:
lq.desc[0]

'1848, West Coast convictions of resourcefulness and persistence, the California Common was born. Easy on the alcohol and weighing in on the taste for a complete balance. Fruit like quality is created by fermenting the lager at ale temperatures. Hints at a full-bodied ale yet retains a crisp finish.'

In [64]:
lq

,name,country,price,sku_link,style,alc,beer_type,desc
0,33 acres of life lager,Canada,$12.29,/product/772335,California Common Or Steam,4.8%,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulness and persistence, the California Common was born. Easy on the alcohol and weighing in on the taste for a complete balance. Fruit like quality is created by fermenting the lager at ale temperatures. Hints at a full-bodied ale yet retains a crisp finish."
1,33 acres of nirvana 6 pack,Canada,$13.49,/product/880930,India Pale Ale,7%,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved craft beer brewers. This West Coast Indian Pale Ale beautifully balances bitter-fresh pine notes with grapefruit rind and candied orange citrus. Try it with some fried chicken for a true taste of nirvana.
2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,Pale Ale,5.6%,Domestic Beer - BC Craft Beer,"A full flavoured beer integrated with a distinct floral hop gives a quality unique to our Pacific Northwest surroundings. This style is evolutionary from a IPA with an assertive hop, lower alcohol content, and slight pine aroma. Low in malt, this beer has a refined quality and a refreshing temperament."
4,33 acres of sunshine,Canada,$21.99,/product/23859,Wheat,5%,Domestic Beer - BC Craft Beer,no info
3,33 acres of sunshine blanche,Canada,$12.29,/product/90837,Wheat,5%,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this romantic French Blanchè can willingly co-star any seasons current scenes. Showcasing flare for the prismatic, 33 Acres of Sunshine synthesizes an unfiltered wheat dream. Joining the arrays, glowing hops of Aurora and Styrian Golding awaken subtle anise seeds, dazzling coriander, and candied orange ovations."
6,alexander keith's - india pale ale,Canada,$14.99,/product/687251,India Pale Ale,5%,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale is light in colour and hopped in flavour. Only the lightest and finest barleys that produce a pale malt are used while the amounts of hops are increased so as to give a pronounced hop flavour."
7,alhambra reserva 1925,Spain,$13.29,/product/638759,Pilsner,6.4%,Import Beer,This award-winning lager is brewed over 35 days. The nose is full-on caramel leading to a full-flavoured palate and a satisfying finish.
8,asahi super dry,Thailand,$14.79,/product/439950,Pilsner,5%,Import Beer,no info
9,asahi super dry big,Japan,$14.79,/product/541201,Pilsner,5%,Import Beer,no info
13,balderdash brewing - sneaky weasel craft lager,Canada,$1.99,/product/198948,Lager,5.6%,Domestic Beer - BC Craft Beer,"Now in a tall can! This mildly hopped craft lager delivers a bold 5.6% alc/vol, but goes down crisp and clean. Pairs well with game day celebrations, fun in the sun and good times with friends."


In [66]:
lq.drop(index = [8,64,173,179,193,194,199,212,213,235,246,261,333,418,426,366], inplace = True) #original index

KeyError: '[  8  64 173 179 193 194 199 212 213 235 246 261 333 418 426 366] not found in axis'

In [67]:
#filter temp1 without row (has desc is "no info") to sort then drop
temp1 = lq[lq.desc != 'no info'].sort_values(['desc','price'], ascending = False).drop_duplicates(subset = ['desc'], keep = 'first')

In [68]:
#add lq 
lq = pd.concat([lq[lq.desc =='no info'],temp1])

In [69]:
lq.sort_index(inplace = True)

In [70]:
lq[lq.desc=='no info']

,name,country,price,sku_link,style,alc,beer_type,desc
4,33 acres of sunshine,Canada,$21.99,/product/23859,Wheat,5%,Domestic Beer - BC Craft Beer,no info
9,asahi super dry big,Japan,$14.79,/product/541201,Pilsner,5%,Import Beer,no info
14,baltika 7,Russia,$2.49,/product/176909,Lager,5.4%,Import Beer,no info
25,belgian moon - white ale 4,Canada,$11.49,/product/183512,Wheat,5.45%,Domestic Beer - National Domestic Beer,no info
31,bomber brewing - skyline lager 6pks,Canada,$12.29,/product/233406,Lager,5%,Domestic Beer - BC Craft Beer,no info
40,bridge brewing - primetime ipa,Canada,$12.49,/product/102114,India Pale Ale,5%,Domestic Beer - BC Craft Beer,no info
43,carib lager,Trinidad And Tobago,$11.99,/product/328419,Lager,5.2%,Import Beer,no info
47,category 12 brewing - chromatic ipa,Canada,$13.29,/product/102035,India Pale Ale,7%,Domestic Beer - BC Craft Beer,no info
49,central city - beer league lager,Canada,$12.99,/product/102622,Lager,5%,Domestic Beer - BC Craft Beer,no info
52,clausthaler premium,Germany,$5.99,/product/722355,no info,0.5%,De-Alcoholized Beer,no info


# Extract lq1

In [304]:
lq1 = lq[lq.beer_type!='Domestic Beer - National Domestic Beer']

In [305]:
def ratio_per_word (word, string):
    ratio = process.extract(word, string, limit = 1, scorer = fuzz.token_set_ratio)

    #sort dictionary by values
    similarity = ratio[0][1] 
    print(similarity)
    beer_comb = ratio[0][0]
    print(beer_comb)
    return beer_comb, similarity
        

In [306]:
long_rate=[]
long_simi = []
for i in lq1.name:
    rate,simi = ratio_per_word(i,beer.comb)
    long_rate.append(rate)
    long_simi.append(simi)

84
33 acres brewing co. 33 acres of life
84
33 acres brewing co. 33 acres of nirvana
79
33 acres brewing co. 33 acres of ocean
83
33 acres brewing co. 33 acres of sunshine
100
33 acres brewing co. 33 acres of sunshine
100
grupo cervezas alhambra, sl reserva 1925
88
asahi breweries ltd asahi super dry
96
balderdash brewing co. sneaky weasel
100
baltika breweries baltika #7 export
65
bard's tale beer company bard's gold
100
barkerville brewing co. 52 foot stout
61
4 mile brewing co. pale ale
51
bard's tale beer company bard's gold
60
swinkels family brewers bavaria beer
86
beau's all natural brewing company beau's - full time ipa tall cans
43
brauerei beck & co. beck's
86
big rock urban brewery and eatery traditional ale
100
bitburger brauerei premium pils (premium beer)
100
bomber brewing park life passion fruit ale
100
bomber brewing skyline lager
67
bowen island brewing co. ltd. coastline pale ale
100
bowen island brewing co. ltd. coastline pale ale
74
granville island brewery west co

In [307]:
lq1['beercomb'] = long_rate
lq1['simi'] = long_simi
lq1['confirm']= np.nan


<ipython-input-307-5a548c47b1ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lq1['beercomb'] = long_rate
<ipython-input-307-5a548c47b1ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lq1['simi'] = long_simi
<ipython-input-307-5a548c47b1ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [308]:
pd.set_option('display.max_colwidth', 50)

In [309]:
lq1['beercomb']=lq1['beercomb'].apply(lambda x: x[0] if len(x)==1 else x)

<ipython-input-309-5c3c73282d94>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lq1['beercomb']=lq1['beercomb'].apply(lambda x: x[0] if len(x)==1 else x)


In [310]:
lq1.loc[ (lq1.simi>=80),'confirm'] ='yes'

/home/sam/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [311]:
lq1.loc[(lq1.simi<70),'confirm'] ='no'

In [312]:
lq1.reset_index(inplace = True)

In [313]:
lq1[(lq1.simi>=70)& (lq1.simi<80)]

,index,name,country,price,sku_link,style,alc,beer_type,desc,beercomb,simi,confirm
2,2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,Pale Ale,5.6%,Domestic Beer - BC Craft Beer,A full flavoured beer integrated with a distin...,33 acres brewing co. 33 acres of ocean,79,NaN
22,35,bowen island - west coast lager,Canada,$8.49,/product/488379,Lager,5%,Domestic Beer - BC Craft Beer,The trees are felled and de-limbed and rolled ...,granville island brewery west coast ipa,74,NaN
25,40,bridge brewing - primetime ipa,Canada,$12.49,/product/102114,India Pale Ale,5%,Domestic Beer - BC Craft Beer,no info,bomber brewing ipa,76,NaN
34,51,central city - red racer session ipa 500 12pk,Canada,$24.29,/product/127109,India Pale Ale,4%,Domestic Beer - BC Craft Beer,Mosaic hops lend to this light and fruity ale ...,central city brewers + distillers red racer ip...,75,NaN
50,80,deschutes - wowza lo-cal hazy pale ale,USA,$14.49,/product/8081,Pale Ale,4%,Import Beer,"With only 100 calories, 4 g of carbs, 4% abv a...",deschutes brewery wowza!,79,NaN
51,81,dos equis xx especial lager,Mexico,$13.99,/product/156174,Lager,4.6%,Import Beer,"Crisp and refreshing up-front. Light body, the...","cervecería cuauhtémoc moctezuma, s.a. de c.v. ...",71,NaN
66,99,fernie brewing - hit the deck hazy ipa,Canada,$12.79,/product/180565,India Pale Ale,6.4%,Domestic Beer - BC Craft Beer,An IPA style made famous by breweries on the E...,bomber brewing ipa,76,NaN
72,106,four winds - nectarous dry hopped sour,Canada,$7.69,/product/871376,Sour,5.5%,Domestic Beer - BC Craft Beer,Cut through carbs with a tart pucker. Four Win...,four winds brewing co. nectarous,78,NaN
77,111,fullers london pride,United Kingdom,$10.49,/product/676767,Pale Ale,4.7%,Import Beer,"Brewed to 4.7%, this tawny-coloured premium al...",fuller smith & turner plc fuller's london pride,75,NaN
80,122,grolsch premium pilsner,Netherlands,$2.29,/product/535294,Lager,5%,Import Beer,Medium gold colour; rich grain and hop aromas ...,strathcona beer company premium pilsner,79,NaN


In [314]:
lq1.iloc[[2,50,51,72,77,94,170,171,175,177,178,179],11] = 'yes'

/home/sam/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [315]:
lq1.loc[lq1.confirm.isna(),'confirm']='no'

In [316]:
lq1.iloc[:,[4,9,10,11]]

,sku_link,beercomb,simi,confirm
0,/product/772335,33 acres brewing co. 33 acres of life,84,yes
1,/product/880930,33 acres brewing co. 33 acres of nirvana,84,yes
2,/product/837591,33 acres brewing co. 33 acres of ocean,79,yes
3,/product/90837,33 acres brewing co. 33 acres of sunshine,83,yes
4,/product/23859,33 acres brewing co. 33 acres of sunshine,100,yes
5,/product/638759,"grupo cervezas alhambra, sl reserva 1925",100,yes
6,/product/541201,asahi breweries ltd asahi super dry,88,yes
7,/product/198948,balderdash brewing co. sneaky weasel,96,yes
8,/product/176909,baltika breweries baltika #7 export,100,yes
9,/product/193590,bard's tale beer company bard's gold,65,no


In [317]:
df = lq.merge(lq1.iloc[:,[4,9,10,11]], on = 'sku_link', how = 'left')

In [318]:
lq = df

In [319]:
pd.set_option('display.max_colwidth', None)

In [320]:
lq.loc[lq.confirm.isna(),'confirm']='no'

In [321]:
lq1['beercomb2'] = long_rate

In [322]:
lq1

,index,name,country,price,sku_link,style,alc,beer_type,desc,beercomb,simi,confirm,beercomb2
0,0,33 acres of life lager,Canada,$12.29,/product/772335,California Common Or Steam,4.8%,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulness and persistence, the California Common was born. Easy on the alcohol and weighing in on the taste for a complete balance. Fruit like quality is created by fermenting the lager at ale temperatures. Hints at a full-bodied ale yet retains a crisp finish.",33 acres brewing co. 33 acres of life,84,yes,33 acres brewing co. 33 acres of life
1,1,33 acres of nirvana 6 pack,Canada,$13.49,/product/880930,India Pale Ale,7%,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved craft beer brewers. This West Coast Indian Pale Ale beautifully balances bitter-fresh pine notes with grapefruit rind and candied orange citrus. Try it with some fried chicken for a true taste of nirvana.,33 acres brewing co. 33 acres of nirvana,84,yes,33 acres brewing co. 33 acres of nirvana
2,2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,Pale Ale,5.6%,Domestic Beer - BC Craft Beer,"A full flavoured beer integrated with a distinct floral hop gives a quality unique to our Pacific Northwest surroundings. This style is evolutionary from a IPA with an assertive hop, lower alcohol content, and slight pine aroma. Low in malt, this beer has a refined quality and a refreshing temperament.",33 acres brewing co. 33 acres of ocean,79,yes,33 acres brewing co. 33 acres of ocean
3,3,33 acres of sunshine blanche,Canada,$12.29,/product/90837,Wheat,5%,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this romantic French Blanchè can willingly co-star any seasons current scenes. Showcasing flare for the prismatic, 33 Acres of Sunshine synthesizes an unfiltered wheat dream. Joining the arrays, glowing hops of Aurora and Styrian Golding awaken subtle anise seeds, dazzling coriander, and candied orange ovations.",33 acres brewing co. 33 acres of sunshine,83,yes,33 acres brewing co. 33 acres of sunshine
4,4,33 acres of sunshine,Canada,$21.99,/product/23859,Wheat,5%,Domestic Beer - BC Craft Beer,no info,33 acres brewing co. 33 acres of sunshine,100,yes,33 acres brewing co. 33 acres of sunshine
5,7,alhambra reserva 1925,Spain,$13.29,/product/638759,Pilsner,6.4%,Import Beer,This award-winning lager is brewed over 35 days. The nose is full-on caramel leading to a full-flavoured palate and a satisfying finish.,"grupo cervezas alhambra, sl reserva 1925",100,yes,"grupo cervezas alhambra, sl reserva 1925"
6,9,asahi super dry big,Japan,$14.79,/product/541201,Pilsner,5%,Import Beer,no info,asahi breweries ltd asahi super dry,88,yes,asahi breweries ltd asahi super dry
7,13,balderdash brewing - sneaky weasel craft lager,Canada,$1.99,/product/198948,Lager,5.6%,Domestic Beer - BC Craft Beer,"Now in a tall can! This mildly hopped craft lager delivers a bold 5.6% alc/vol, but goes down crisp and clean. Pairs well with game day celebrations, fun in the sun and good times with friends.",balderdash brewing co. sneaky weasel,96,yes,balderdash brewing co. sneaky weasel
8,14,baltika 7,Russia,$2.49,/product/176909,Lager,5.4%,Import Beer,no info,baltika breweries baltika #7 export,100,yes,baltika breweries baltika #7 export
9,15,bards tale - gold gluten free lager,USA,$15.29,/product/193590,Gluten-free,4.7%,Import Beer,"100% gluten-free lager brewed with malted sorghum provides a traditional beer flavour. It has a medium clear, amber/gold colour, moderate head, aromas of roasted malt and toasted grains with sweet fruit notes. It's dry and light-bodied with balanced hops and flavours of apple, caramel and sweet grain.",bard's tale beer company bard's gold,65,no,bard's tale beer company bard's gold


In [323]:
lq[lq.confirm =='no']

,name,country,price,sku_link,style,alc,beer_type,desc,beercomb,simi,confirm
5,alexander keith's - india pale ale,Canada,$14.99,/product/687251,India Pale Ale,5%,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale is light in colour and hopped in flavour. Only the lightest and finest barleys that produce a pale malt are used while the amounts of hops are increased so as to give a pronounced hop flavour.",NaN,NaN,no
10,bards tale - gold gluten free lager,USA,$15.29,/product/193590,Gluten-free,4.7%,Import Beer,"100% gluten-free lager brewed with malted sorghum provides a traditional beer flavour. It has a medium clear, amber/gold colour, moderate head, aromas of roasted malt and toasted grains with sweet fruit notes. It's dry and light-bodied with balanced hops and flavours of apple, caramel and sweet grain.",bard's tale beer company bard's gold,65.0,no
12,barkerville - sluice juice pale ale,Canada,$12.79,/product/204407,Pale Ale,5%,Domestic Beer - BC Craft Beer,"Amassing a bounty of apricot, peach and tangerine aromas, Sluice Juice is a hazy, golden juice bomb that rewards your taste buds with riches of fruit forward flavor.",4 mile brewing co. pale ale,61.0,no
13,baron bohemian tmavy lezak,Czech Republic,$2.19,/product/127282,Dark Lager,4.7%,Import Beer,Baron Bohemian Beer pours a dark red brown hue with toasty caramel flavours on the palate and hints of dark chocolate; crisp and firmly crafted in the Bohemian style.,bard's tale beer company bard's gold,51.0,no
14,bavaria original brew,Netherlands,$13.29,/product/676437,Pilsner,5%,Import Beer,"Bavaria Premium Pilsner is one of the best-known beers, loved for its extremely pure taste. It has a beautiful, bright yellow golden colour and a lightly flavoured aroma with floral hop notes, but it is not too bitter. Bavaria Premium Pilsner is made from 100% natural pure mineral water.",swinkels family brewers bavaria beer,60.0,no
16,becks,Germany,$13.79,/product/9209,Pilsner,5%,Import Beer,A Pilsner style lager with a distinctive and refreshing taste. Beck's is brewed in accordance with the German Purity Law of 1516.,brauerei beck & co. beck's,43.0,no
17,belgian moon - mango wheat,Canada,$11.49,/product/101624,Wheat,5.4%,Domestic Beer - National Domestic Beer,"A cloudy yellow-orange appearance with a rich, white head. Aromas of mango and light honeyed-biscuits, with ripe mango flavours and a biscuity malt with hints of clove and honey. Finishes qucik and dry with some mango notes lingering.",NaN,NaN,no
18,belgian moon - white ale 4,Canada,$11.49,/product/183512,Wheat,5.45%,Domestic Beer - National Domestic Beer,no info,NaN,NaN,no
23,bowen island - artisan ipa,Canada,$8.99,/product/12757,India Pale Ale,5.5%,Domestic Beer - BC Craft Beer,"The shipyard is alive with the sounds of artisans at their task. The boat wood's floral and citrus fragrances swirl through a solid framework, planed smooth and perfectly balanced. A rich golden colour sets off the final details. Artisans build it best.",bowen island brewing co. ltd. coastline pale ale,67.0,no
25,bowen island - west coast lager,Canada,$8.49,/product/488379,Lager,5%,Domestic Beer - BC Craft Beer,"The trees are felled and de-limbed and rolled into the inlet. The logs are bound together and driven through the crisp waters of the strait. A mellow timber aroma drifts with the raft, mixing with the refreshing sea breeze. Float with a Westcoast Lager.",granville island brewery west coast ipa,74.0,no
